<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

## 1 Загрузка данных

Загрузим данные и посмотрим на них.

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропущенных значений нет. Тип данных столбцов "Возраст" и "Зарплата" определился как float. Возможно, по каким-то причинам в таблице появились нецелые значения. Если это не так, то преобразуем тип в int. Если найдутся дробные, посмотрим на них.

In [4]:
def to_int (column_name):
    if (abs(data[column_name] - np.round(data[column_name])) >= 0.1).sum() == 0: 
        #Количество строк, в которых после запятой не 0
        
        data[column_name] = data[column_name].astype('int')
        
to_int('Возраст')
to_int('Зарплата')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null int64
Зарплата             5000 non-null int64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: int64(5)
memory usage: 195.4 KB


Дробных значений нет, тип данных преобразован.

In [5]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.359400,1.194200,0.148000
std,0.500049,8.440807,9900.082063,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Отрицательных или неожиданно больших значений в таблице нет. Мужчин и женщин примерно поровну. Средние и медианные значения возраста и зарплат примерно равны, в данных нет ощутимого сдвига вправо или влево.

**Вывод**

Данные загружены и изучены. В таблице нет пропусков или неожиданных значений. 

Тип данных столбцов "Возраст" и "Зарплата" преобразован в int.

## 2 Умножение матриц

Умножим матрицу признаков на обратимую матрицу и посмотрим, как это повлияет на предсказания линейной регрессии.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

После умножения $XP$ предсказания модели будут следующими: $a_{p} = XPw_{p}$, где $w_{p} = ((XP)^T XP)^{-1} (XP)^T y$. 

Подставим значение $w_{p}$ в формулу для расчёта предсказаний и преобразуем выражение с использованием свойств матриц:

- $(AB)^T = B^T A^T$
- $(AB)^{-1} = B^{-1} A^{-1}$

$a_{p} = XP((XP)^T XP)^{-1} (XP)^T y = XP(P^T X^T XP)^{-1} P^T X^T y = XPP^{-1}(P^T X^T X)^{-1} P^T X^T y = XE(X^T X)^{-1} (P^T)^{-1} P^T X^T y = X(X^T X)^{-1} E X^T y = X(X^T X)^{-1} X^T y = Xw = a$

После преобразования предсказания линейной регрессии не изменились.

**Ответ:** Если матрицу признаков $X$ умножить на обратимую матрицу $P$, качество линейной регрессии не изменится.

$$w_{p} = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w$$

$$a_{p} = XPw_{p} = a$$

**Обоснование:** Качество линейной регрессии определяется её предсказаниями. Если предсказания одинаковые, то и любые посчитанные метрики будут одинаковыми.

## 3 Алгоритм преобразования

**Алгоритм**

В нашей таблице четыре признака и 5000 объектов, плюс столбец с единицами - получается матрица $X$ имеет размер $5000\times 5$.

Создадим случайную обратимую матрицу $P_{5\times 5}$ и умножим на неё $X$. Получившаяся матрица $XP$ будет того же размера, что и $X$.

Если значение $P$ неизвестны, восстановить $X$ из $XP$ будет практически невозможно. При этом, зная $P$, достаточно умножить $XP$ на $P^{-1}$ для получения $X$.

**Обоснование**

Как показано в разделе 2, умножение матрицы признаков на обратимую матрицу не влияет на качество линейной регрессии. Поэтому можно зашифровать $X$ с помощью случайно сгенерированной матрицы $P_{5\times 5}$.

## 4 Проверка алгоритма

Сохраним матрицу признаков и вектор целевого признака.

In [6]:
X = data.drop('Страховые выплаты', axis=1).values
X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1) #Добавляем столбец с единицами
y = data['Страховые выплаты'].values

X.shape

(5000, 5)

Сгенерируем случайную обратимую матрицу $P_{5\times 5}$ и сохраним значение $P^{-1}$ для восстановления $X$ при необходимости.

In [7]:
P_size = X.shape[1]

np.random.seed(42)
P = np.random.random((P_size, P_size))
try:
    P_1 = np.linalg.inv(P)
except:
    print('Невероятно! Сгенерировалась необратимая матрица!')

Получим новую зашифрованную матрицу признаков.

In [8]:
XP = X.dot(P)

XP.shape

(5000, 5)

Убедимся, что преобразование не повлияло на качество линейной регрессии. Обучим модель на исходных данных, а затем на преобразованных и сравним значения R2.

In [9]:
model = LinearRegression()
model.fit(X, y)
print('До преобразования R2 = {:.3f}'.format(r2_score(y, model.predict(X))))

До преобразования R2 = 0.425


In [10]:
model.fit(XP, y)
print('После преобразования R2 = {:.3f}'.format(r2_score(y, model.predict(XP))))

После преобразования R2 = 0.425


Преобразование не повлияло на модель, данные успешно обезличены.

## 5 Вывод

Данные загружены и проанализированы, типы данных преобразованы. 

Предложен алгоритм шифрования данных - умножение на случайную обратимую матрицу. Доказано, что такое преобразование не влияет на качество предсказаний линейной регрессии.

Данные зашфрованы по предложенному алгоритму. Показано, что качество линейной регрессии после шифрования не изменилось.

## 6 Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования